GSE169159: blood, healthy afaik, longitudinal 
GSE171964: Single-cell data, deseased, PBMC

Preprocessing bulk: 
- outlier detection with PCA (3 samples removed as they were more than 1.5sd away)
raw counts available 
- RLE > 0.6 removed 
- normalised by DESeq2 / BTMsT8

Relative log expression (RLE) plots were generated with EDAseq40; samples with an RLE > 0.6 were removed from the analysis. Differential gene analysis was performed using DESeq241 (v.1.26.0), incorporating participant identifier into the model to account for inter-participant bias. Genes were ranked by the Wald statistic as reported by DESeq2 for GSEA using the BTMs18. Per-participant fold changes were computed by dividing the DESeq2 normalized expression data for the day of interest by either day 0 (for day 1, day2 and day 7) or day 21 (for day 22, 28 and 42). To obtain BTM correlates with age, the age of each participant was compared against the per-participant fold changes for day 22. The resulting correlation values were ranked by t-statistic and analysed with GSEA. The same method was employed to obtain BTM correlates with IFNγ. IFN scores were computed by taking the per-participant mean fold change on day 22 of the unique set of genes present in the 5 interferon BTMs (M75, M111.1, M150, M127 and M68) that significantly correlated with day-22 IFNγ fold change. Similarly, the per-participant M16 gene score was computed using average fold change on day 22 of the genes present in M16.


Check: GSE97590 -> normalized count matrix from GEO used... -> we see this paper also does not compare microarray data with RNseq
(Baseline normalized log2-transformed fold changes were then computed per subject for all genes. GSEA was then performed to identify enriched BTMs using gene lists for each dataset ranked by t-statistic from two-sided Student’s t-tests on the post-vaccination log2-transformed fold changes.)




In [ ]:
import pandas as pd
from filepath import *

In [ ]:
#With his I mostly want to find out how i can integrate this papers data into ImmPort
# TODO find out what is _BL (i suspect Baseline): Check it means baseline, i gutesss i will take this for the GAMLSS

In [ ]:
## Testwise integration of 2020_Arachandra_Wimmers

metadata_df = pd.read_csv(PATH_ARUNACHALAM2021_METADATA)
print(metadata_df.shape)
metadata_df.head()

In [ ]:
counts_df = pd.read_csv(PATH_ARUNACHALAM2021_RAWDATA, index_col='gene')
print(counts_df.shape)
counts_df.head()

In [ ]:

# Problem: SRA Toolkit metadata does not have the study Subject accessions ...  Let´s see how else i can get them 
import sys
import importlib
import os
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt
importlib.reload(geo_utils)

metadata_geo_df = read_prefixes_from_metadata_txt(PATH_ARUNACHALAM2021_GEOMETADATA) # here we get age and sex info but my method yet overwrite it with the last of that tag (here it was !Sample_characteristics_ch1)
print(metadata_geo_df.shape)
## As this meta file saves several infos under the same "tag" I manually need to rename the cols to get desired info


metadata_geo_df = metadata_geo_df.rename(columns={'characteristics_ch1':'age', 
                                'characteristics_ch1.2':'day', 
                                'characteristics_ch1.3': 'sex'})
metadata_geo_df['age'] = metadata_geo_df['age'].str.lstrip('age: ')
metadata_geo_df['day'] = metadata_geo_df['day'].str.lstrip('day: ')
metadata_geo_df['sex'] = metadata_geo_df['sex'].str.lstrip('Sex: ')

metadata_geo_df.head()

In [ ]:
counts_df_T = counts_df.T
combi_df = counts_df.merge(metadata_geo_df, right_on='title', how='left', left_on='gene')
print(combi_df.shape)
combi_df.head()

In [ ]:
combi_df.to_csv('2021_aranachalam_vaccine_combi_df.csv')

In [ ]:
## Now I want to have only the Baseline data, as we dont need all the longituninal stuff and also thats not a healthy anymore
counts_df_two = counts_df
counts_df_two_filtered = counts_df_two.filter(regex='_BL$')

counts_df_two_filtered_T = counts_df_two_filtered.T
combi_df_baselines = counts_df_two_filtered_T.merge(metadata_geo_df, right_on='title', how='left', left_index=True)
meta_cols = list(combi_df_baselines.columns[-37:])
count_cols = list(combi_df_baselines.columns[:-37])
new_order_cols = meta_cols + count_cols
combi_df_baselines = combi_df_baselines[new_order_cols]
print(combi_df_baselines.shape)
combi_df_baselines.head()

In [ ]:
combi_df_baselines.to_csv(PATH_ARUNACHALAM2021_FINALDESTINATION)